In [ ]:
import csv
import math
def rawSD(close_prices):
    mean = sum(close_prices) / len(close_prices)
    diff = [(close_prices[i] - mean) ** 2 for i in range(len(close_prices))]
    varience = sum(diff) / (len(close_prices))
    sd = math.sqrt(varience)
    return sd


In [84]:
with open('BTC-USD.csv',mode = 'r') as file:
    btc_usd = csv.DictReader(file)

#     PARKINSON
    arr = [i for i in btc_usd]    
    arr = arr[0:-2]
    h2l = [math.log(float(arr[i]['High'])/float(arr[i]['Low']))**2 for i in range(len(arr))]
#    last *len(h2l) is doubted in varience, it is not present in sheet https://docs.google.com/spreadsheets/d/1lQTz1Fad2TIi42-jBjwd-42UWKsIuSP1/edit#gid=1351734694
    parkinson_varience = (sum(h2l)/len(h2l))/4*math.log(2)
    parkinson_volatility = math.sqrt(parkinson_varience)
    print('parkinson_volatility:',parkinson_volatility)
    
#     KLASS
    c2o = [math.log(float(arr[i]['Close'])/float(arr[i]['Open']))**2 for i in range(len(arr))]
    gk_varience = [h2l[i]*0.5 + c2o[i]*(2*math.log(2) - 1) for i in range(len(c2o))]
    garman_klass = math.sqrt(sum(gk_varience)/len(gk_varience))
    print('garman_klass:',garman_klass)
    
    
#     Yang Zhang
    o = [float(arr[i]['Open'])/float(arr[i-1]['Close']) for i in  range(1, len(arr))]
    next_open_sd = rawSD(o)
    c2o_sd = rawSD(c2o)
    n = len(arr)-1
    k = 0.34/(1.34+((n+1)/(n-1)))
    u = [math.log(float(arr[i]['High'])/float(arr[i]['Open']))**2 for i in range(len(arr))]
    d = [math.log(float(arr[i]['Low'])/float(arr[i]['Open']))**2 for i in range(len(arr))]
    rs_arr = [u[i]*(u[i] - c2o[i]) + d[i]*(d[i] - c2o[i]) for i in range(len(u))]
    rs = math.sqrt(sum(rs_arr)/(n+1))
    yang_zhang = math.sqrt(next_open_sd**2 + k**2 + (1-k)*(rs**2))
    print('yang_zhang:',yang_zhang)
    


parkinson_volatility: 0.01728304559885097
garman_klass: 0.03396928577908559
yang_zhang: 0.14496684530571033
